This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [12]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '...'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [13]:
# First, import the relevant modules
import requests

In [14]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sample = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-03&end_date=2017-01-02&' + API_KEY)

In [15]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample.json()

{u'errors': {u'start_date': [u'start_date should not exceed end_date']},
 u'quandl_error': {u'code': u'QESx04',
  u'message': u'You have submitted incorrect query parameters. Please check your API call syntax and try again.'}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [16]:
# Task 1
data = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=_jXRx-TTsCsrsNCHU5SN')

In [17]:
# Task 2
afx_x = data.json()

In [18]:
# Task 3

# First, we need to know the indexes of the 'Open' price in the stock entry for any given day
idx_open = afx_x['dataset']['column_names'].index('Open')

# We need to populate a new list of opening prices
# The data for each day resides in the 'data' key, which is a list of lists
# Each nested list contains all the different values for a single day
opening_prices = []
for lst in afx_x['dataset']['data']:
    opening_prices.append(lst[idx_open])

# We could also do this via a list comprehension
opening_prices = [lst[idx_open] for lst in afx_x['dataset']['data']]

# Values returned from the API can sometimes be 'None', so we need to remove those
opening_prices = [price for price in opening_prices if price is not None]

# Finally, getting the highest and lowest price is trivial
highest_opening_price = max(opening_prices)
lowest_opening_price = min(opening_prices)

In [19]:
# Task 4

# We need the indexes for the high and low prices
idx_high = afx_x['dataset']['column_names'].index('High')
idx_low = afx_x['dataset']['column_names'].index('Low')

# Initialize a placeholder result variable
largest_change = 0

# Calculate the largest change
for lst in afx_x['dataset']['data']:
    if lst[idx_high] - lst[idx_low] > largest_change:
        largest_change = lst[idx_high] - lst[idx_low]

In [20]:
# Task 5

# Find the index for the Closing Price
idx_close = afx_x['dataset']['column_names'].index('Close')

# Populate a list of Closing Price differences
closing_diffs = []
for i in range(1, len(afx_x['dataset']['data'])):
    diff = afx_x['dataset']['data'][i][idx_close] - afx_x['dataset']['data'][i-1][idx_close]
    closing_diffs.append(abs(diff))
    
# Calculate the maximum difference
max_closing_diff = max(closing_diffs)

In [21]:
# Task 6

# Find the index of the Traded Volume
idx_volume = afx_x['dataset']['column_names'].index('Traded Volume')

# Populate a list of the Traded Volumes
traded_volumes = [lst[idx_volume] for lst in afx_x['dataset']['data']]

# Find the average daily volume
sum(traded_volumes)/len(traded_volumes)

89124.33725490196

In [22]:
# Task 7 (Optional)

# Answers will vary